--Configuración del NLTK--

In [ ]:
# Importando librerías necesarias
import nltk
import os
import random

# Configurar la ruta para los datos de NLTK
nltk_data_path = os.path.join(os.getcwd(), 'nltk_data')

# Crear el directorio si no existe
os.makedirs(nltk_data_path, exist_ok=True)

# Descargar recursos necesarios de NLTK en la ruta especificada
nltk.download('punkt', download_dir=nltk_data_path)
nltk.download('punkt_tab', download_dir=nltk_data_path)

# Añadir la ruta a la configuración de NLTK
nltk.data.path.append(nltk_data_path)

from nltk.tokenize import word_tokenize

--Creación de los diccionarios de los intents y responses--

In [ ]:
# Diccionario de intenciones
intents = {
    "saludo": ["hola", "buenas", "qué tal"],
    "despedida": ["adios", "chao", "hasta luego"],
    "ayuda": ["ayuda", "duda", "pregunta"],
}

# Diccionario de respuestas (contiene múltiples respuestas para cada intención)
responses = {
    "saludo": [ 
        "¡Hola! Bienvenido a ElectroStore, ¿en qué puedo ayudarte?",
        "¡Buenas! ¿En qué puedo ayudarte?",
        "¡Hey! Gracias por visitarnos. ¿Qué buscas hoy?"
    ],
    "despedida": [
        "¡Gracias por visitarnos! Que tengas un buen día",
        "¡Hasta luego! No dudes en volver si tienes más preguntas.",
        "¡Chao! Esperamos verte pronto."
    ],
    "ayuda": [
        "Estoy aquí para ayudarte. Puedes preguntarme por horarios, envíos o productos.",
        "No dudes en preguntar cualquier duda que tengas.",
        "Estoy a tu disposición para resolver tus inquietudes."
    ],
}

--Ampliación de intenciones y respuestas--

In [ ]:
intents.update({
    "horario": ["horario", "abren", "cierran", "horas"],
    "envio": ["envío", "entrega", "coste envío", "gastos envío"],
    "producto": ["producto", "artículo", "disponible", "stock"],
})

responses.update({
    "horario": [
        "Nuestro horario de atención es de lunes a viernes de 9:00 a 18:00.",
        "Estamos abiertos de lunes a viernes de 9:00 a 18:00.",
        "Puedes visitarnos de lunes a viernes de 9:00 a 18:00."
    ],
    "envio": [
        "El coste de envío depende de la ubicación y el peso del paquete.",
        "Ofrecemos envío gratuito en pedidos superiores a 50€.",
        "Los gastos de envío se calculan al finalizar la compra."
    ],
    "producto": [
        "Todos nuestros productos están disponibles en nuestra tienda online.",
        "Puedes consultar la disponibilidad de un artículo en nuestra web.",
        "Si un producto no está en stock, puedes solicitar que te avisemos cuando vuelva a estar disponible."
    ]
})

--Implementación del Stemming/Lematización--

In [ ]:
# Función para procesar la entrada del usuario y devolver una respuesta
from nltk.stem import SnowballStemmer

# Inicializar el stemmer para español
stemmer = SnowballStemmer('spanish')

--Peso de las intenciones: más alto = más prioridad--

In [ ]:
intents_weights = {
    "saludo": 0.5,
    "despedida": 0.5,
    "ayuda": 0.5,
    "horario": 1,
    "envio": 1,
    "producto": 1,
}

--Manejo de la conversación--

In [ ]:
# Variable global para el contexto
context = None

# Función para manejar la conversación
def chatbot(user_input):
    global context
    # Tokenizar la entrada del usuario y aplicar stemming
    tokens = [stemmer.stem(t) for t in word_tokenize(user_input.lower())]
    
    best_intent = None # Intención con más coincidencias
    max_score = 0    # Número máximo de coincidencias
    
    # Buscar intenciones en la entrada del usuario
    for intent, keywords in intents.items():
        stemmed_keywords = [stemmer.stem(k) for k in keywords]
        # Contar cuántas palabras clave coinciden
        matches = sum(1 for token in tokens if token in stemmed_keywords)
        # Guardar la intención con más coincidencias, ponderada por su peso
        score = matches * intents_weights.get(intent, 1)
        if score > max_score:
            best_intent = intent
            max_score = score

    if best_intent is None and context is not None:
        # Si no se encuentra una intención, usar el contexto anterior
        best_intent = context

    if best_intent is not None and max_score > 0:
        context = best_intent  # Actualizar el contexto
        
    if best_intent:
        # Seleccionar una respuesta aleatoria de las posibles respuestas
        return random.choice(responses[best_intent])
    else:
        # Respuesta por defecto si no se encuentra ninguna intención
        return "Lo siento, no entiendo tu pregunta. ¿Puedes reformularla?"

--Simulación de la conversación--

In [ ]:
# Simulación de la conversación
# Texto de bienvenida
print("Chatbot: ¡Hola! Soy tu asistente de ElectroStore. Escribe 'salir' para terminar.")
# Bucle de conversación
while True:
    # Entrada del usuario
    user = input("Tú: ")
    # Condición de salida
    if user.lower() in ['salir', 'exit', 'quit']:
        # Mensaje de despedida
        print("Chatbot: ¡Gracias por visitarnos! Que tengas un buen día.")
        # Salir del bucle
        break
    # Obtener respuesta del chatbot
    response = chatbot(user)
    # Mostrar respuesta del chatbot
    print(f"Chatbot: {response}")

--Simulación de convesación con la interfaz ipywidgets--

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Área de chat
chat_area = widgets.Output(layout=widgets.Layout(border='1px solid black', height='400px', overflow_y='auto', padding='10px'))

# Cuadro de entrada
input_box = widgets.Text(
    value='',                                      # Valor inicial vacío
    placeholder='Escribe tu pregunta aquí...',     # Placeholder
    description='Tú:',                             # Etiqueta
    disabled=False,                                # No deshabilitado                     
    layout=widgets.Layout(width='100%')            # Ancho completo
)

# Función para agregar mensajes al área de chat
def add_message(user_input, bot_response):
    with chat_area:
        # Mensaje del usuario
        display(HTML(f"""
        <div style="text-align:right; margin:5px;">
            <span style="
                background-color:#007bff;
                color:white;
                padding:8px 12px;
                border-radius:15px;
                display:inline-block;
                max-width:70%;
                word-wrap:break-word;
            ">{user_input}</span>
        </div>
        """))
        
        # Respuesta del bot
        display(HTML(f"""
        <div style="text-align:left; margin:5px;">
            <span style="
                background-color:#28a745;
                color:white;
                padding:8px 12px;
                border-radius:15px;
                display:inline-block;
                max-width:70%;
                word-wrap:break-word;
            ">{bot_response}</span>
        </div>
        """))

# Función para manejar la entrada del usuario
def on_enter(text):
    user_input = text.value
    if user_input.strip() == '':
        return # Ignorar entradas vacías
    text.value = ''  # Limpiar el cuadro de entrada
    bot_response = chatbot(user_input)
    add_message(user_input, bot_response)

input_box.on_submit(on_enter)

display(chat_area, input_box)